# Where do the water masses go in the Salish Sea?
Using the same water mass seperations as in CIOPS

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.interpolate as interp
# from scipy.stats import binned_statistic_2d
# import cmocean.cm as cm
# from matplotlib.patches import Rectangle
# from matplotlib.collections import PatchCollection

In [2]:
def get_data(mydata, starttime, endtime):
    salt = mydata.init_salt[(mydata.init_t >= starttime) & (mydata.init_t <= endtime)]
    temp = mydata.init_temp[(mydata.init_t >= starttime) & (mydata.init_t <= endtime)]
    depth = depthf(mydata.init_z[(mydata.init_t >= starttime) & (mydata.init_t <= endtime)]-1.)[0]
    section = mydata.final_section[(mydata.init_t >= starttime) & (mydata.init_t <= endtime)]
    trans = mydata.init_transp[(mydata.init_t >= starttime) & (mydata.init_t <= endtime)]/(endtime-starttime+1)
    return salt, temp, depth, section, trans

In [3]:
mymesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask201702.nc')
depthf = interp.interp1d(mymesh.z, mymesh.gdept_1d)

In [4]:
# section numbers
meander = 0
admiralty = 2
deception = 3
rosario = 4
sanjuan = 5
haro = 6
sec_name = ['meander', 'admiralty', 'deception', 'rosario', 'sanjuan', 'haro']

## summer 2017

In [20]:
yearjumps = [0,1,-2,1,0,1,0,1,1,0,1,0]
str_mo = ['jan', 'feb', 'mar', 'apr','may', 'jun', 'jul', 'aug', 'sep','oct', 'nov', 'dec']
sum_start = 1+(5*720+(sum(yearjumps[:5+1]*24)))
sum_end = 720+(8*720+(sum(yearjumps[:8+1]*24)))

file = '/ocean/rbeutel/MOAD/analysis-becca/Ariane/1yr_runs/201905_1hr/forward_01jan17/ariane_positions_quantitative.nc'
mydata = xr.open_dataset(file)
salt, temp, depth, section, trans = get_data(mydata, sum_start, sum_end)

In [23]:
d = {'salt': salt, 'temp': temp, 'depth':depth, 'section':section, 'transport':trans}
df = pd.DataFrame(data=d)

In [25]:
upwelled = df[df.depth <= 200]
cuc = df[df.depth > 200]

In [30]:
[np.sum(upwelled.transport)/np.sum(df.transport)*100,np.sum(cuc.transport)/np.sum(df.transport)*100]

[93.73213662963585, 6.267863370364149]

In [39]:
# lets see what sections these groups get into 
# first upwelled
print('advects back out = ' + str(sum(upwelled.transport[upwelled.section == meander])/np.sum(upwelled.transport)*100))
print('haro = ' + str(sum(upwelled.transport[upwelled.section == haro])/np.sum(upwelled.transport)*100))
print('sanjuan = ' + str(sum(upwelled.transport[upwelled.section == sanjuan])/np.sum(upwelled.transport)*100))
print('rosario = ' + str(sum(upwelled.transport[upwelled.section == rosario])/np.sum(upwelled.transport)*100))
print('admiralty = ' + str(sum(upwelled.transport[upwelled.section == admiralty])/np.sum(upwelled.transport)*100))
print('deception = ' + str(sum(upwelled.transport[upwelled.section == deception])/np.sum(upwelled.transport)*100))

advects back out = 81.38937375212231
haro = 13.237420502107383
sanjuan = 0.7064967587767096
rosario = 1.4683930037920245
admiralty = 3.1828024570276363
deception = 0.015434759321449222


In [40]:
# and deep
print('advects back out = ' + str(sum(cuc.transport[cuc.section == meander])/np.sum(cuc.transport)*100))
print('haro = ' + str(sum(cuc.transport[cuc.section == haro])/np.sum(cuc.transport)*100))
print('sanjuan = ' + str(sum(cuc.transport[cuc.section == sanjuan])/np.sum(cuc.transport)*100))
print('rosario = ' + str(sum(cuc.transport[cuc.section == rosario])/np.sum(cuc.transport)*100))
print('admiralty = ' + str(sum(cuc.transport[cuc.section == admiralty])/np.sum(cuc.transport)*100))
print('deception = ' + str(sum(cuc.transport[cuc.section == deception])/np.sum(cuc.transport)*100))

advects back out = 88.46222790645251
haro = 8.610096977372251
sanjuan = 0.4545295871380527
rosario = 0.8177067747539721
admiralty = 1.6480737956203904
deception = 0.007364958662847872


## Winter 2016/17

In [41]:
yearjumps = [0,1,-2,1,0,1,0,1,1,0,1,0]
str_mo = ['jan', 'feb', 'mar', 'apr','may', 'jun', 'jul', 'aug', 'sep','oct', 'nov', 'dec']
win_start = 1+(9*720+(sum(yearjumps[:9+1]*24)))
win_end = 720+(11*720+(sum(yearjumps[:11+1]*24)))

file = '/ocean/rbeutel/MOAD/analysis-becca/Ariane/1yr_runs/201905_1hr/forward_01jan16/ariane_positions_quantitative.nc'
mydata = xr.open_dataset(file)
saltW, tempW, depthW, sectionW, transW = get_data(mydata, win_start, win_end)

win_start = 1+(0*720+(sum(yearjumps[:0+1]*24)))
win_end = 720+(5*720+(sum(yearjumps[:5+1]*24)))

file = '/ocean/rbeutel/MOAD/analysis-becca/Ariane/1yr_runs/201905_1hr/forward_01jan17/ariane_positions_quantitative.nc'
mydata = xr.open_dataset(file)
saltS, tempS, depthS, sectionS, transS = get_data(mydata, win_start, win_end)

salt = np.append(saltW, saltS)
temp = np.append(tempW, tempS)
depth = np.append(depthW, depthS)
section = np.append(sectionW, sectionS)
trans = np.append(transW, transS)

In [42]:
d = {'salt': salt, 'temp': temp, 'depth':depth, 'section':section, 'transport':trans}
df = pd.DataFrame(data=d)

In [44]:
upwelled = df[(df.depth <= 200) & (df.salt >= 32.4)]
cuc = df[(df.depth > 200) & (df.salt >= 32.4)]
columbia = df[(df.salt < 32.4) & (df.temp >= 10.7)]
coldF = df[(df.salt < 32.4) & (df.temp < 10.7)]

In [45]:
[np.sum(upwelled.transport)/np.sum(df.transport)*100,np.sum(cuc.transport)/np.sum(df.transport)*100,
np.sum(columbia.transport)/np.sum(df.transport)*100,np.sum(coldF.transport)/np.sum(df.transport)*100]

[53.93843695128463, 6.166366356170525, 12.21886924572361, 27.67632744682128]

In [46]:
# lets see what sections these groups get into 
# first upwelled
print('advects back out = ' + str(sum(upwelled.transport[upwelled.section == meander])/np.sum(upwelled.transport)*100))
print('haro = ' + str(sum(upwelled.transport[upwelled.section == haro])/np.sum(upwelled.transport)*100))
print('sanjuan = ' + str(sum(upwelled.transport[upwelled.section == sanjuan])/np.sum(upwelled.transport)*100))
print('rosario = ' + str(sum(upwelled.transport[upwelled.section == rosario])/np.sum(upwelled.transport)*100))
print('admiralty = ' + str(sum(upwelled.transport[upwelled.section == admiralty])/np.sum(upwelled.transport)*100))
print('deception = ' + str(sum(upwelled.transport[upwelled.section == deception])/np.sum(upwelled.transport)*100))

advects back out = 84.82765618044542
haro = 10.052585541846364
sanjuan = 0.5810074713520935
rosario = 1.0484117680470646
admiralty = 3.259665938453609
deception = 0.011176476026704643


In [47]:
# and deep
print('advects back out = ' + str(sum(cuc.transport[cuc.section == meander])/np.sum(cuc.transport)*100))
print('haro = ' + str(sum(cuc.transport[cuc.section == haro])/np.sum(cuc.transport)*100))
print('sanjuan = ' + str(sum(cuc.transport[cuc.section == sanjuan])/np.sum(cuc.transport)*100))
print('rosario = ' + str(sum(cuc.transport[cuc.section == rosario])/np.sum(cuc.transport)*100))
print('admiralty = ' + str(sum(cuc.transport[cuc.section == admiralty])/np.sum(cuc.transport)*100))
print('deception = ' + str(sum(cuc.transport[cuc.section == deception])/np.sum(cuc.transport)*100))

advects back out = 93.48854029970028
haro = 4.305421294243852
sanjuan = 0.260153862976561
rosario = 0.4206077366042773
admiralty = 1.3420340829031492
deception = 0.0028524863047357826


In [48]:
# columbia 
print('advects back out = ' + str(sum(columbia.transport[columbia.section == meander])/np.sum(columbia.transport)*100))
print('haro = ' + str(sum(columbia.transport[columbia.section == haro])/np.sum(columbia.transport)*100))
print('sanjuan = ' + str(sum(columbia.transport[columbia.section == sanjuan])/np.sum(columbia.transport)*100))
print('rosario = ' + str(sum(columbia.transport[columbia.section == rosario])/np.sum(columbia.transport)*100))
print('admiralty = ' + str(sum(columbia.transport[columbia.section == admiralty])/np.sum(columbia.transport)*100))
print('deception = ' + str(sum(columbia.transport[columbia.section == deception])/np.sum(columbia.transport)*100))

advects back out = 95.46860207507646
haro = 2.5278198648165646
sanjuan = 0.18915306618356834
rosario = 0.2884615409518763
admiralty = 1.5074949537499662
deception = 0.0029119133068299546


In [49]:
# and myterious colder fresh that may still be columbia
print('advects back out = ' + str(sum(coldF.transport[coldF.section == meander])/np.sum(coldF.transport)*100))
print('haro = ' + str(sum(coldF.transport[coldF.section == haro])/np.sum(coldF.transport)*100))
print('sanjuan = ' + str(sum(coldF.transport[coldF.section == sanjuan])/np.sum(coldF.transport)*100))
print('rosario = ' + str(sum(coldF.transport[coldF.section == rosario])/np.sum(coldF.transport)*100))
print('admiralty = ' + str(sum(coldF.transport[coldF.section == admiralty])/np.sum(coldF.transport)*100))
print('deception = ' + str(sum(coldF.transport[coldF.section == deception])/np.sum(coldF.transport)*100))

advects back out = 97.60974873439106
haro = 1.413398694821987
sanjuan = 0.1055362301996183
rosario = 0.1611048628052375
admiralty = 0.6685709219620392
deception = 0.0019854090112765596


## Winter 2017/18

In [5]:
yearjumps = [0,1,-2,1,0,1,0,1,1,0,1,0]
str_mo = ['jan', 'feb', 'mar', 'apr','may', 'jun', 'jul', 'aug', 'sep','oct', 'nov', 'dec']
win_start = 1+(9*720+(sum(yearjumps[:9+1]*24)))
win_end = 720+(11*720+(sum(yearjumps[:11+1]*24)))

file = '/ocean/rbeutel/MOAD/analysis-becca/Ariane/1yr_runs/201905_1hr/forward_01jan17/ariane_positions_quantitative.nc'
mydata = xr.open_dataset(file)
salt17, temp17, depth17, section17, trans17 = get_data(mydata, win_start, win_end)

start = 1
end = 720

file = '/ocean/rbeutel/MOAD/analysis-becca/Ariane/201905_1hr/forward_01jan18/ariane_positions_quantitative.nc'
mydata = xr.open_dataset(file)
salt_jan, temp_jan, depth_jan, section_jan, trans_jan = get_data(mydata, start, end)

file = '/ocean/rbeutel/MOAD/analysis-becca/Ariane/201905_1hr/forward_01feb18/ariane_positions_quantitative.nc'
mydata = xr.open_dataset(file)
salt_feb, temp_feb, depth_feb, section_feb, trans_feb = get_data(mydata, start, end)

file = '/ocean/rbeutel/MOAD/analysis-becca/Ariane/201905_1hr/forward_01mar18/ariane_positions_quantitative.nc'
mydata = xr.open_dataset(file)
salt_mar, temp_mar, depth_mar, section_mar, trans_mar = get_data(mydata, start, end)

salt = np.append(np.append(np.append(salt17, salt_jan), salt_feb), salt_mar)
temp = np.append(np.append(np.append(temp17, temp_jan), temp_feb), temp_mar)
depth = np.append(np.append(np.append(depth17, depth_jan), depth_feb), depth_mar)
section = np.append(np.append(np.append(section17, section_jan), section_feb), section_mar)
trans = np.append(np.append(np.append(trans17, trans_jan), trans_feb), trans_mar)

In [6]:
d = {'salt': salt, 'temp': temp, 'depth':depth, 'section':section, 'transport':trans}
df = pd.DataFrame(data=d)

In [16]:
upwelled = df[(df.depth <= 200) & (df.salt >= 31.8) & (df.temp <= 9.8)]
cuc = df[(df.depth > 200) & (df.salt >= 31.8) & (df.temp <= 9.8)]
columbia = df[(df.temp > 9.8)]
coldF = df[(df.salt < 31.8) & (df.temp < 9.8)]

In [17]:
[np.sum(upwelled.transport)/np.sum(df.transport)*100,np.sum(cuc.transport)/np.sum(df.transport)*100,
np.sum(columbia.transport)/np.sum(df.transport)*100,np.sum(coldF.transport)/np.sum(df.transport)*100]

[67.65680137196978, 6.095397136901457, 2.910667440202735, 23.337134050926053]

In [10]:
# lets see what sections these groups get into 
# first upwelled
print('advects back out = ' + str(sum(upwelled.transport[upwelled.section == meander])/np.sum(upwelled.transport)*100))
print('haro = ' + str(sum(upwelled.transport[upwelled.section == haro])/np.sum(upwelled.transport)*100))
print('sanjuan = ' + str(sum(upwelled.transport[upwelled.section == sanjuan])/np.sum(upwelled.transport)*100))
print('rosario = ' + str(sum(upwelled.transport[upwelled.section == rosario])/np.sum(upwelled.transport)*100))
print('admiralty = ' + str(sum(upwelled.transport[upwelled.section == admiralty])/np.sum(upwelled.transport)*100))
print('deception = ' + str(sum(upwelled.transport[upwelled.section == deception])/np.sum(upwelled.transport)*100))

advects back out = 84.48769768837384
haro = 9.512216135860179
sanjuan = 0.5948117849499793
rosario = 0.949235722815569
admiralty = 3.2630895839510474
deception = 0.011524260302600666


In [11]:
# and deep
print('advects back out = ' + str(sum(cuc.transport[cuc.section == meander])/np.sum(cuc.transport)*100))
print('haro = ' + str(sum(cuc.transport[cuc.section == haro])/np.sum(cuc.transport)*100))
print('sanjuan = ' + str(sum(cuc.transport[cuc.section == sanjuan])/np.sum(cuc.transport)*100))
print('rosario = ' + str(sum(cuc.transport[cuc.section == rosario])/np.sum(cuc.transport)*100))
print('admiralty = ' + str(sum(cuc.transport[cuc.section == admiralty])/np.sum(cuc.transport)*100))
print('deception = ' + str(sum(cuc.transport[cuc.section == deception])/np.sum(cuc.transport)*100))

advects back out = 92.6043680365625
haro = 4.5736470339224535
sanjuan = 0.2979765798597585
rosario = 0.47534916550493256
admiralty = 1.3799345621068348
deception = 0.0064622023368504766


In [12]:
# columbia 
print('advects back out = ' + str(sum(columbia.transport[columbia.section == meander])/np.sum(columbia.transport)*100))
print('haro = ' + str(sum(columbia.transport[columbia.section == haro])/np.sum(columbia.transport)*100))
print('sanjuan = ' + str(sum(columbia.transport[columbia.section == sanjuan])/np.sum(columbia.transport)*100))
print('rosario = ' + str(sum(columbia.transport[columbia.section == rosario])/np.sum(columbia.transport)*100))
print('admiralty = ' + str(sum(columbia.transport[columbia.section == admiralty])/np.sum(columbia.transport)*100))
print('deception = ' + str(sum(columbia.transport[columbia.section == deception])/np.sum(columbia.transport)*100))

advects back out = 92.74943674918838
haro = 3.521841332016041
sanjuan = 0.262490034609221
rosario = 0.456335502099864
admiralty = 2.941398331565594
deception = 0.003391871578109074


In [13]:
# and myterious colder fresh that may still be columbia
print('advects back out = ' + str(sum(coldF.transport[coldF.section == meander])/np.sum(coldF.transport)*100))
print('haro = ' + str(sum(coldF.transport[coldF.section == haro])/np.sum(coldF.transport)*100))
print('sanjuan = ' + str(sum(coldF.transport[coldF.section == sanjuan])/np.sum(coldF.transport)*100))
print('rosario = ' + str(sum(coldF.transport[coldF.section == rosario])/np.sum(coldF.transport)*100))
print('admiralty = ' + str(sum(coldF.transport[coldF.section == admiralty])/np.sum(coldF.transport)*100))
print('deception = ' + str(sum(coldF.transport[coldF.section == deception])/np.sum(coldF.transport)*100))

advects back out = 97.33664742878084
haro = 1.3138447959238753
sanjuan = 0.09233904049191399
rosario = 0.18328809458081916
admiralty = 0.8120089115428383
deception = 0.0027974182462729125
